<a href="https://colab.research.google.com/github/ricoen/google-ee-data-extraction/blob/main/ee_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=6F0Y_UkL7W-tzh6b2jEypLagtQyWpymQSjyfdYzmkV4&tc=ZeYg5T1k21EOKyPJZlXPQhLmt4WNPwyfMQMq-_q5JLY&cc=poN3VLh0a8VOPpZNdgYlH3bDvMDnZDRbFnuLOeGyaZI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh62G9wxg4hXyMQeOS43XMvbZ0wIhclt3Dc-MurA8unD5tWzqyd1vQk

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [77]:
dataset = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')

In [78]:
start_date = "2022-07-01"
stop_date = "2022-07-31"

dataset = dataset.select("Tair_f_inst").filterDate(start_date, stop_date)

In [79]:
lon = 112.611523
lat = -7.951988

poi = ee.Geometry.Point(lon, lat)

In [80]:
res = 27830

t_air = dataset.mean().sample(poi, res).first().get('Tair_f_inst').getInfo()
print('Average daytime air termperature at urban point:', round(t_air -273.15, 2), '°C')

Average daytime air termperature at urban point: 20.55 °C


In [81]:
t_air_poi = dataset.getRegion(poi, res).getInfo()

In [82]:
t_air_poi[:3]

[['id', 'longitude', 'latitude', 'time', 'Tair_f_inst'],
 ['A20220701_0000',
  112.6255151784935,
  -7.875036022469579,
  1656633600000,
  292.84552001953125],
 ['A20220701_0300',
  112.6255151784935,
  -7.875036022469579,
  1656644400000,
  298.3895263671875]]

In [83]:
import pandas as pd
from datetime import datetime as dt

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    time = dataset.aggregate_array("system:time_start").getInfo()
    date = [x // 1000 for x in time]
    captured = [dt.utcfromtimestamp(i).strftime("%c GMT") for i in date]
    captured = pd.Series(captured)
    # df['datetime'] = pd.to_datetime(df['time'], unit='ms')
    df["datetime"] = pd.to_datetime(captured).dt.tz_convert("Asia/Jakarta")

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [84]:
t_air_df = ee_array_to_df(t_air_poi, ['Tair_f_inst'])

In [85]:
t_air_df.head()

,time,datetime,Tair_f_inst
0,1656633600000,2022-07-01 07:00:00+07:00,292.845520
1,1656644400000,2022-07-01 10:00:00+07:00,298.389526
2,1656655200000,2022-07-01 13:00:00+07:00,297.594788
3,1656666000000,2022-07-01 16:00:00+07:00,295.278717
4,1656676800000,2022-07-01 19:00:00+07:00,291.940063


In [86]:
def t_air_to_celsius(t_air):
    """Converts MODIS LST units to degrees Celsius."""
    t_celsius =  t_air - 273.15
    
    return round(t_celsius)

t_air_df['Tair_f_inst'] = t_air_df['Tair_f_inst'].apply(t_air_to_celsius)

In [87]:
t_air_df.head()

,time,datetime,Tair_f_inst
0,1656633600000,2022-07-01 07:00:00+07:00,20
1,1656644400000,2022-07-01 10:00:00+07:00,25
2,1656655200000,2022-07-01 13:00:00+07:00,24
3,1656666000000,2022-07-01 16:00:00+07:00,22
4,1656676800000,2022-07-01 19:00:00+07:00,19


In [88]:
# from datetime import datetime as dt

# t_air_df["datetime"] = t_air_df["datetime"].dt.tz_convert("Asia/Jakarta")

In [91]:
t_air_df

,time,datetime,Tair_f_inst
0,1656633600000,2022-07-01 07:00:00+07:00,20
1,1656644400000,2022-07-01 10:00:00+07:00,25
2,1656655200000,2022-07-01 13:00:00+07:00,24
3,1656666000000,2022-07-01 16:00:00+07:00,22
4,1656676800000,2022-07-01 19:00:00+07:00,19
...,...,...,...
235,1659171600000,2022-07-30 16:00:00+07:00,23
236,1659182400000,2022-07-30 19:00:00+07:00,19
237,1659193200000,2022-07-30 22:00:00+07:00,18
238,1659204000000,2022-07-31 01:00:00+07:00,18


In [90]:
#Visualize the data with Plotly
import plotly.express as px

#Define x and y values
fig = px.line(t_air_df, x="datetime", y="Tair_f_inst", 
              title="Air Temperature on Malang City - GLDAS-2.1 (NASA)")
#Show the figure
fig.show()